In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [30]:
with open('notMNIST_sanit.pickle','rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save # for save memory
    print ('training',train_dataset.shape,train_labels.shape)
    print ('testing',test_dataset.shape,test_labels.shape)
    print ('validation',valid_dataset.shape,valid_labels.shape)

training (200000, 28, 28) (200000,)
testing (8676, 28, 28) (8676,)
validation (8933, 28, 28) (8933,)


Reformat into a shape that's more adapted to the models we're going to train:

data as a flat matrix

labels as float 1-hot encodings.

In [35]:
# reshape
train_dataset = np.reshape(train_dataset,(train_dataset.shape[0],28*28))
valid_dataset = np.reshape(valid_dataset,(valid_dataset.shape[0],28*28))
test_dataset = np.reshape(test_dataset,(test_dataset.shape[0],28*28))


print ('train',train_dataset.shape,'valid',valid_dataset.shape,'test',test_dataset.shape)

train (200000, 784) valid (8933, 784) test (8676, 784)


In [22]:
# put label through one-hot-encoder
# there is something called broastcast in numpy.......

label_feature = np.array([0,1,4,2,4,0,3])

label_pool = np.arange(5)


# add a new axis to label_feature
labels[:,None].shape

label_encoder = (label_pool == labels[:,None]).astype(np.float32)
print (label_encoder)

[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.]]


In [31]:
def one_hot_encoder(label_set):
    label_new = (np.arange(10) == label_set[:,None]).astype(np.float32)
    return label_new

train_labels = one_hot_encoder(train_labels)
test_labels = one_hot_encoder(test_labels)
valid_labels = one_hot_encoder(valid_labels)

print ('train_labels', train_labels.shape)
print ('test_labels', test_labels.shape)
print ('valid_labels', valid_labels.shape)

train_labels (200000, 10)
test_labels (8676, 10)
valid_labels (8933, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:

First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

with graph.as_default():

...

Then you can run the operations on this graph as many times as you want by calling session.run(), providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:
with tf.Session(graph=graph) as session:

...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [38]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
image_size = 28
num_label = 10

graph = tf.Graph()
#all the construction in netural network, all the element in this graph
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset,])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #define the variable 
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_label]))
    #why biases use 
    biases = tf.Variable(tf.zeros([num_label]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # know here only use train_dataset
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
      tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [41]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
      tf.initialize_all_variables().run()
      print('Initialized')
      for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
          print('Loss at step %d: %f' % (step, l))
          print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
          # Calling .eval() on valid_prediction is basically like calling run(), but
          # just to get that one numpy array. Note that it recomputes all its graph
          # dependencies.
          print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.700408
Training accuracy: 9.0%
Validation accuracy: 13.5%
Loss at step 100: 2.298953
Training accuracy: 71.6%
Validation accuracy: 69.3%
Loss at step 200: 1.833258
Training accuracy: 74.7%
Validation accuracy: 71.7%
Loss at step 300: 1.587299
Training accuracy: 76.0%
Validation accuracy: 72.6%
Loss at step 400: 1.425670
Training accuracy: 77.0%
Validation accuracy: 73.1%
Loss at step 500: 1.306295
Training accuracy: 77.4%
Validation accuracy: 73.4%
Loss at step 600: 1.212705
Training accuracy: 77.8%
Validation accuracy: 73.6%
Loss at step 700: 1.136812
Training accuracy: 78.5%
Validation accuracy: 73.8%
Loss at step 800: 1.073787
Training accuracy: 79.2%
Validation accuracy: 73.8%
Test accuracy: 81.2%


In [44]:
batch_size = 128
num_labels = 10
graph = tf.Graph()
with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = tf.matmul(tf_train_dataset, weights) + biases
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [45]:
num_steps = 3001

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.510942
Minibatch accuracy: 17.2%
Validation accuracy: 8.5%
Minibatch loss at step 500: 1.253512
Minibatch accuracy: 76.6%
Validation accuracy: 74.5%
Minibatch loss at step 1000: 1.348142
Minibatch accuracy: 76.6%
Validation accuracy: 75.5%
Minibatch loss at step 1500: 0.843183
Minibatch accuracy: 78.9%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 0.897752
Minibatch accuracy: 82.8%
Validation accuracy: 76.4%
Minibatch loss at step 2500: 0.979265
Minibatch accuracy: 73.4%
Validation accuracy: 77.1%
Minibatch loss at step 3000: 1.066841
Minibatch accuracy: 78.9%
Validation accuracy: 77.6%
Test accuracy: 85.2%


Problem


Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() 


and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [51]:
batch_size = 128
num_labels = 10
num_hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
      biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
      weights2 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
      biases2 = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
      logits = tf.matmul(lay1_train, weights2) + biases2
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
      valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
      lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
      test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

In [52]:
num_steps = 3001

with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 346.163452
Minibatch accuracy: 9.4%
Validation accuracy: 31.7%
Minibatch loss at step 500: 20.139090
Minibatch accuracy: 78.9%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 12.329384
Minibatch accuracy: 79.7%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 8.222973
Minibatch accuracy: 85.2%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 1.764799
Minibatch accuracy: 88.3%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 3.480980
Minibatch accuracy: 83.6%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 2.148890
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Test accuracy: 88.6%
